In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("cnn_dailymail","3.0.0")

In [3]:
dataset['train']

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [4]:
dataset['train'][5]['article'][:200]

"BAGHDAD, Iraq (CNN) -- Dressed in a Superman shirt, 5-year-old Youssif held his sister's hand Friday, seemingly unaware that millions of people across the world have been touched by his story. Nearby,"

In [5]:
dataset['train'][5]['highlights']

'Parents beam with pride, can\'t stop from smiling from outpouring of support .\nMom: "I was so happy I didn\'t know what to do"\nBurn center in U.S. has offered to provide treatment for reconstructive surgeries .\nDad says, "Anything for Youssif"'

In [6]:
from transformers import pipeline

In [7]:
import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

0


In [8]:
#Creating the pipeline
pipe = pipeline("text-generation",model="gpt2-medium",device=device)

In [9]:
#Preparing the data and inserting into pip
dataset['train'][5]['article'][:2500]
input_text = dataset['train'][5]['article'][:2500]
query = input_text + "\nTL;DR:\n"
pipe_out = pipe(query,max_length=1024,clean_up_tokenization_spaces=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [10]:
summary = pipe_out[0]['generated_text'][len(query):]

In [11]:
all_summary ={}
all_summary['gpt2-medium'] = summary
print(f"Summary :{summary}")

Summary :The Youssif family has been receiving hundreds of thousands in donations to help. It's hard to explain how important this whole thing really is. I don't know how I feel about the Youssif's story, but I'm glad people across the nation -- even the one world I live in -- are stepping by their help, and sharing it. You can tell people have been touched, in terms of how people are helping donate their money.  I wish more people felt the same way, and that even people who'd never heard of this story wouldn't feel ashamed about giving money -- even in the U.S.


In [12]:
#Trying out the google pegasus model
pipe = pipeline('summarization',device=device,model='google/pegasus-cnn_dailymail')

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
result = pipe(input_text)

In [14]:
all_summary['pegasus_cnn'] = result[0]['summary_text']

In [15]:
for model in all_summary:
    print(model.upper())
    print(all_summary[model])
    print("")

GPT2-MEDIUM
The Youssif family has been receiving hundreds of thousands in donations to help. It's hard to explain how important this whole thing really is. I don't know how I feel about the Youssif's story, but I'm glad people across the nation -- even the one world I live in -- are stepping by their help, and sharing it. You can tell people have been touched, in terms of how people are helping donate their money.  I wish more people felt the same way, and that even people who'd never heard of this story wouldn't feel ashamed about giving money -- even in the U.S.

PEGASUS_CNN
Youssif's mother: "I was so happy I didn't know what to do with myself"<n>Boy's father: "We just want to thank everyone who has come forward"<n>Youssif will undergo surgery in the United States .<n>You can make a donation at the Children's Burn Foundation's site .



In [16]:
#Trying with custom text
input_text = "Neelam Shinde, who hails from Maharashtra's Satara district, was critically injured in the accident on February 14 and has been in a coma at a hospital since then. Neelam Shinde has reportedly suffered severe injuries to her head, hand, and chest. Her family has sought an urgent visa to travel to the US to be by her side.The MEA has taken up the matter with the US.The US side is looking into the formalities for early grant of visa for the applicant's family,the sources cited in the PTI report said.Nationalist Congress Party (Sharadchandra Pawar) leader Supriya Sule flagged the case on Wednesday.Student Neelam Shinde has met with an accident in the USA and is hospitalised in a local hospital, she said in a post on X."

In [17]:
result = pipe(input_text)
print(f"Summary with pegasus_cnn: {result[0]['summary_text']}")

Summary with pegasus_cnn: Neelam Shinde has reportedly suffered severe injuries to her head, hand, and chest .<n>Her family has sought an urgent visa to travel to the US to be by her side .<n>The US side is looking into the formalities for early grant of visa for the applicant's family .
